In [7]:
import shutil
import os
import time
import schedule
import git
import sys
import subprocess
import requests
import socket
'''def diagnose_network():
    """全面诊断网络连接问题"""
    print("=== 网络连接诊断 ===")
    
    # 测试基本连接
    try:
        socket.create_connection(('github.com', 443), timeout=10)
        print("✓ 可以连接到 github.com:443")
    except socket.error as e:
        print(f"✗ 无法连接到 github.com:443: {e}")
    
    # 测试HTTP访问
    try:
        response = requests.get('https://github.com', timeout=10)
        print(f"✓ GitHub网站可访问，状态码: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"✗ GitHub网站访问失败: {e}")
    
    # 测试DNS解析
    try:
        ip = socket.gethostbyname('github.com')
        print(f"✓ DNS解析成功: github.com -> {ip}")
    except socket.gaierror as e:
        print(f"✗ DNS解析失败: {e}")
    
    # 检查代理设置
    print("\n=== 代理设置检查 ===")
    env_vars = ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']
    for var in env_vars:
        if var in os.environ:
            print(f"{var}: {os.environ[var]}")
        else:
            print(f"{var}: 未设置")
    
    # 检查Git配置
    print("\n=== Git配置检查 ===")
    try:
        result = subprocess.run(['git', 'config', '--global', '--get', 'http.proxy'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Git HTTP代理: {result.stdout.strip()}")
        else:
            print("Git HTTP代理: 未设置")
            
        result = subprocess.run(['git', 'config', '--global', '--get', 'https.proxy'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Git HTTPS代理: {result.stdout.strip()}")
        else:
            print("Git HTTPS代理: 未设置")
    except Exception as e:
        print(f"Git配置检查失败: {e}")'''

def backupLocal(path='..\\backup'):
    files=[file for file in os.listdir('.') if os.path.isfile(file)]
    if not os.path.exists(path):
        os.makedirs(path)
    date=time.strftime('%Y%m%d%H%M%S')
    os.makedirs(f'{path}\\{date}')
    for file in files:
        shutil.copy(file,f'{path}\\{date}\\{file}')

def backupGitHub(path='C:\\Users\\gsr00\\软工\\code'):
    try:
        repo=git.Repo(path)
    except:
        repo.git.Repo.init(path)
    new_url='https://github.com/gsr0003/code.git'
    origin=repo.remote('origin')
    origin.set_url(new_url)
    repo.git.add('-A')
    date=time.strftime('%Y.%m.%d %H:%M:%S')
    commit_message=f'Automatically update at {date}'
    repo.index.commit(commit_message)
    print("开始推送...")
    origin.push()
    print('推送成功')

'''def backupGitHub(path='C:\\Users\\gsr00\\软工\\code'):
    """使用系统git命令获得更详细的错误信息"""
    
    try:
        original_cwd = os.getcwd()
        os.chdir(path)
        
        print("当前目录:", os.getcwd())
        
        # 添加文件
        result_add = subprocess.run(
            ['git', 'add', '-A'],
            capture_output=True,
            text=True,
            timeout=30
        )
        if result_add.returncode != 0:
            print(f"添加文件失败: {result_add.stderr}")
            return False
        
        # 提交
        date = time.strftime('%Y.%m.%d %H:%M:%S')
        commit_message = f'Automatically update at {date}'
        result_commit = subprocess.run(
            ['git', 'commit', '-m', commit_message],
            capture_output=True,
            text=True,
            timeout=30
        )
        if result_commit.returncode != 0:
            print(f"提交失败: {result_commit.stderr}")
            # 可能是没有更改，继续尝试推送
            
        # 推送 - 使用详细输出
        print("开始推送...")
        process = subprocess.Popen(
            ['git', 'push', '--verbose'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True
        )
        
        # 实时输出
        while True:
            output = process.stdout.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                print(output.strip())
                sys.stdout.flush()  # 立即刷新输出
        
        return_code = process.poll()
        os.chdir(original_cwd)
        
        if return_code == 0:
            print('推送成功')
            return True
        else:
            print(f'推送失败，返回码: {return_code}')
            return False
            
    except subprocess.TimeoutExpired:
        print('操作超时')
        os.chdir(original_cwd)
        return False
    except Exception as e:
        print(f'备份失败: {e}')
        os.chdir(original_cwd)
        return False'''

'''def backupGitHub(path='C:\\Users\\gsr00\\软工\\code'):
    """尝试不同的网络配置"""
    import subprocess
    
    original_cwd = os.getcwd()
    os.chdir(path)
    
    try:
        # 方法1: 取消所有代理设置
        print("尝试方法1: 取消代理设置...")
        env = os.environ.copy()
        # 移除代理环境变量
        for proxy_var in ['HTTP_PROXY', 'HTTPS_PROXY', 'http_proxy', 'https_proxy']:
            env.pop(proxy_var, None)
        
        # 取消Git代理配置
        subprocess.run(['git', 'config', '--global', '--unset', 'http.proxy'], 
                      capture_output=True)
        subprocess.run(['git', 'config', '--global', '--unset', 'https.proxy'], 
                      capture_output=True)
        
        result = subprocess.run(['git', 'push'], env=env, timeout=60)
        if result.returncode == 0:
            print("推送成功!")
            return True
        
        # 方法2: 使用SSH替代HTTPS
        print("尝试方法2: 切换到SSH协议...")
        subprocess.run(['git', 'remote', 'set-url', 'origin', 'git@github.com:gsr0003/code.git'])
        result = subprocess.run(['git', 'push'], timeout=60)
        if result.returncode == 0:
            print("SSH推送成功!")
            return True
            
        # 方法3: 使用更短的超时和重试
        print("尝试方法3: 使用短超时和重试...")
        for attempt in range(3):
            try:
                result = subprocess.run(
                    ['git', 'push'], 
                    timeout=30,  # 30秒超时
                    capture_output=True,
                    text=True
                )
                if result.returncode == 0:
                    print("推送成功!")
                    return True
                else:
                    print(f"尝试 {attempt+1} 失败: {result.stderr}")
            except subprocess.TimeoutExpired:
                print(f"尝试 {attempt+1} 超时")
            
            if attempt < 2:
                print("等待5秒后重试...")
                time.sleep(5)
        
        print("所有方法都失败")
        return False
        
    except Exception as e:
        print(f"备份失败: {e}")
        return False
    finally:
        os.chdir(original_cwd)'''

if __name__=='__main__':
    #diagnose_network()
    backupGitHub()


开始推送...


GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git push --porcelain -- origin
  stderr: 'fatal: unable to access 'https://github.com/gsr0003/code.git/': Failed to connect to github.com port 443 after 21079 ms: Could not connect to server'